In [2]:
import os
import json
for file in os.listdir('./resources/Redacted files'):
	content = ""
	filename = None
	with open(f'./resources/Redacted files/{file}', 'r') as f:
		data = json.load(f)

		for data_point in data:
			content += data_point['text']

			if filename is None:
				filename = data_point['metadata']['file_name'].split('.')[0]
		
	content = content.replace('\n\n', '\n')
	with open(f'./resources/cleaned/{filename}.md', 'w') as f:
		f.write(content)

----
## *OLD*

### Building Knowledge Base

In [14]:
tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [32]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=150,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

### Parsing Docs

In [4]:
import json

with open("./resources/Redacted files/AUTONOMOUS ROBOT Evaluation Report (No Award) 170122_redacted.json", "r") as f:
	data = (json.load(f))

texts = [doc['text'] for doc in data]

### Create embeddings

In [11]:
from FlagEmbedding import FlagModel
model = FlagModel('BAAI/bge-large-en-v1.5', use_fp16=True, normalize_embeddings=True)

c:\Users\Name\anaconda3\envs\gpt4v\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 366/366 [00:00<?, ?B/s] 
c:\Users\Name\anaconda3\envs\gpt4v\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Name\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administr

In [39]:
df = []
for text in texts:
	chunks = text_splitter.split_text(text)
	for chunk in chunks:
		df.append([chunk, model.encode(chunk)])

df = pd.DataFrame(df, columns = ['for_embedding', 'embedding'])